In [1]:
from transformers import AutoTokenizer
import datasets
import requests

In [40]:
sst2 = datasets.load_dataset("/root/group-shared/jrc/data/sst2", "default")

In [2]:
instruction = "In this task, you are given sentences from movie reviews. The task is to classify a sentence as \"positive\" if the sentiment of the sentence is positive or as \"negative\" if the sentiment of the sentence is negative."
system_prompt = ""
generate_model_ip = "127.0.0.1"
generate_model_port = 33337
generate_model_config = {
    "temperature": 0.0,
    "max_new_tokens": 1,
    "custom_params": {"token_id_list": [30487, 42224]}
}
tokenizer = AutoTokenizer.from_pretrained("/root/group-shared/jrc/base_models/Qwen3-32B")

In [59]:
from typing import Any

from sglang.srt.sampling.custom_logit_processor import CustomLogitProcessor

class DeterministicLogitProcessor(CustomLogitProcessor):
    """A dummy logit processor that changes the logits to always
    sample the given token id.
    """

    def __call__(self, logits, custom_param_list):
        # Check that the number of logits matches the number of custom parameters
        assert logits.shape[0] == len(custom_param_list)
        key = "token_id_list"

        for i, param_dict in enumerate(custom_param_list):
            # Mask all other tokens
            temp_logits = logits[i, :].clone()
            logits[i, :] = -float("inf")
            logits[i, param_dict[key]] = temp_logits[param_dict[key]]
        return logits

In [60]:
example = sst2["train"][10]
print(example["sentence"])
print(example["label"])

goes to absurd lengths 
0


In [61]:
prompt = tokenizer.apply_chat_template(
    [{"role": "system", "content": system_prompt}, {"role": "user", "content": f"{instruction}\nText: {example['sentence']}"}], 
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)
response = requests.post(
    f"http://{generate_model_ip}:{generate_model_port}/generate",
    json={
        "text": prompt,
        "sampling_params": generate_model_config,
        "custom_logit_processor": DeterministicLogitProcessor().to_str(),
    },
)
completion = response.json()["text"]
print(completion)

negative


In [32]:
x = tokenizer.encode("ent")
print(x)
x = tokenizer.encode("contr")
print(x)
x = tokenizer.encode("neutral")
print(x)
x = tokenizer.encode("not")
print(x)


[306]
[8222]
[59568]
[1921]
